In [148]:
from pathlib import Path
import json

import pandas as pd
import bids
from datalad import api
from nilearn import image
import numpy as np
pd.set_option('max_colwidth',400)

In [206]:
# setup git
# setup git
! git config --global user.email "you@example.com"
! git config --global user.name "Your Name"

#first clone openneuro and install the datasets, this takes a little while (~20 minutes)
api.clone('///openneuro', '../data/openneuro')
api.install('../data/openneuro', dataset='../data/openneuro', recursive=True)

In [ ]:
api.clone('///openneuro', path = ../data/

In [3]:
oneuro = Path('../data/openneuro/')

In [13]:
rests = sorted(oneuro.glob('**/*_task-rest_*run-*_bold.nii.gz'))

In [64]:
entities = pd.DataFrame([bids.layout.parse_file_entities(rr) for rr in rests])
entities['dataset'] = [rr.parts[3] for rr in rests]
entities['path'] = rests

# clean up entities a little
entities = entities.loc[(entities.acquisition != 'localizer') 
                        & (entities.desc != 'preproc')
                        & ~entities.path.apply(lambda x: 'derivatives' in x.as_posix())
                        & ~entities.path.apply(lambda x: x.parts[-1][:2] == '._')]

In [65]:
nruns = entities.groupby('dataset')[['run']].nunique()
multi_run_ds = nruns.query('run > 1').index.values

In [66]:
mre = entities.loc[entities.dataset.isin(multi_run_ds)]

In [68]:
# inspect to make sure there aren't any weird paths here
mre.groupby('dataset').first().path.values

array([PosixPath('../data/openneuro/ds000031/sub-01/ses-002/func/sub-01_ses-002_task-rest_run-001_bold.nii.gz'),
       PosixPath('../data/openneuro/ds000133/sub-01/ses-post/func/sub-01_ses-post_task-rest_run-01_bold.nii.gz'),
       PosixPath('../data/openneuro/ds000221/sub-010001/ses-02/func/sub-010001_ses-02_task-rest_acq-AP_run-01_bold.nii.gz'),
       PosixPath('../data/openneuro/ds000243/sub-001/func/sub-001_task-rest_run-1_bold.nii.gz'),
       PosixPath('../data/openneuro/ds001168/sub-01/ses-1/func/sub-01_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz'),
       PosixPath('../data/openneuro/ds001386/sub-01/func/sub-01_task-rest_run-1_bold.nii.gz'),
       PosixPath('../data/openneuro/ds001553/sub-001/ses-01/func/sub-001_ses-01_task-rest_run-001_bold.nii.gz'),
       PosixPath('../data/openneuro/ds001875/sub-01/func/sub-01_task-rest_run-01_bold.nii.gz'),
       PosixPath('../data/openneuro/ds002105/sub-02/ses-rest/func/sub-02_ses-rest_task-rest_run-1_bold.nii.gz'),
       PosixP

In [86]:
# get the first bold image from each dataset
first_images = mre.groupby('dataset').first()
first_images.groupby('dataset').first().path.apply(lambda x: api.get(x, dataset='/'.join(x.parts[:3]), on_failure='ignore'))

action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
action summary:
  get (notneeded: 2)
get(error): ds003658/sub-01/func/sub-01_task-rest_run-1_bold.nii.gz (file) [download failed: Not Found
download failed: Not Found
download failed: Not Found
download failed: Not Found
do

dataset
ds000031    [{'action': 'get', 'type': 'dataset', 'path': ...
ds000133    [{'action': 'get', 'type': 'dataset', 'path': ...
ds000221    [{'action': 'get', 'type': 'dataset', 'path': ...
ds000243    [{'action': 'get', 'type': 'dataset', 'path': ...
ds001168    [{'action': 'get', 'type': 'dataset', 'path': ...
ds001386    [{'action': 'get', 'type': 'dataset', 'path': ...
ds001553    [{'action': 'get', 'type': 'dataset', 'path': ...
ds001875    [{'action': 'get', 'type': 'dataset', 'path': ...
ds002105    [{'action': 'get', 'type': 'dataset', 'path': ...
ds002307    [{'action': 'get', 'type': 'dataset', 'path': ...
ds002330    [{'action': 'get', 'type': 'dataset', 'path': ...
ds002372    [{'action': 'get', 'type': 'dataset', 'path': ...
ds002551    [{'action': 'get', 'type': 'dataset', 'path': ...
ds002735    [{'action': 'get', 'type': 'dataset', 'path': ...
ds002898    [{'action': 'get', 'type': 'dataset', 'path': ...
ds003011    [{'action': 'get', 'type': 'dataset', 'path': ...


In [115]:
# pull out the dimensions of the images

dims = []
pixdims = []
for x in first_images.path:
    if not x.exists():
        dims.append(np.ones(8) * np.nan)
        pixdims.append(np.ones(8) * np.nan)
        continue
    ximg = image.load_img(x.as_posix())
    dims.append(ximg.header['dim'])
    pixdims.append(ximg.header['pixdim'])
dims = np.array(dims)
pixdims = np.array(pixdims)
first_images['x_res'] = pixdims[:, 1]
first_images['y_res'] = pixdims[:, 2]
first_images['z_res'] = pixdims[:, 3]
first_images['tr'] = pixdims[:, 4]
first_images['time'] = pixdims[:, 4] * dims[:, 4]/60

## Now we've got some potential datasets, let's look through and see if any are good

In [156]:
potential_ds = first_images.loc[(pixdims[:, [1, 2, 3]]<= 2).prod(1) == 1]
potential_ds

,subject,session,task,run,suffix,datatype,extension,echo,acquisition,space,desc,direction,path,x_res,y_res,z_res,tr,time
dataset,,,,,,,,,,,,,,,,,,
ds000031,01,002,rest,1,bold,func,.nii.gz,None,None,None,None,None,../data/openneuro/ds000031/sub-01/ses-002/func/sub-01_ses-002_task-rest_run-001_bold.nii.gz,2.000000,2.000000,1.998,1.2,8.960000
ds001168,01,1,rest,1,bold,func,.nii.gz,None,fullbrain,None,None,None,../data/openneuro/ds001168/sub-01/ses-1/func/sub-01_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz,1.500000,1.500000,1.500,3000.0,15000.000000
ds001875,01,None,rest,1,bold,func,.nii.gz,None,None,None,None,None,../data/openneuro/ds001875/sub-01/func/sub-01_task-rest_run-01_bold.nii.gz,1.000000,1.000000,1.100,1.0,10.000000
ds002330,01,None,rest,1,bold,func,.nii.gz,None,None,None,None,None,../data/openneuro/ds002330/sub-01/func/sub-01_task-rest_run-01_bold.nii.gz,1.500000,1.500000,1.500,1.9,4.750000
ds002372,RTC01,func01,rest,1,bold,func,.nii.gz,None,None,None,None,None,../data/openneuro/ds002372/sub-RTC01/ses-func01/func/sub-RTC01_ses-func01_task-rest_run-01_bold.nii.gz,2.000000,2.000000,2.000,1.0,6.416667
ds003059,001,LSD,rest,1,bold,func,.nii.gz,None,None,None,None,None,../data/openneuro/ds003059/sub-001/ses-LSD/func/sub-001_ses-LSD_task-rest_run-01_bold.nii.gz,2.000000,2.000000,2.000,2.0,7.233333
ds003520,CTL3692,1,rest,1,bold,func,.nii.gz,None,EPI,None,None,None,../data/openneuro/ds003520/sub-CTL3692/ses-1/func/sub-CTL3692_ses-1_task-rest_acq-EPI_run-1_bold.nii.gz,0.359375,0.359375,1.000,1.5,10.000000
ds003646,HDRA02C,None,rest,1,bold,func,.nii.gz,None,te15,None,None,None,../data/openneuro/ds003646/sub-HDRA02C/func/sub-HDRA02C_task-rest_acq-te15_run-01_bold.nii.gz,0.400000,0.400000,0.400,2.0,5.000000
ds003673,pa1372,None,rest,1,bold,func,.nii.gz,None,None,None,None,None,../data/openneuro/ds003673/sub-pa1372/func/sub-pa1372_task-rest_run-01_bold.nii.gz,2.000000,2.000000,2.000,1.0,6.833333


In [162]:
def print_ds_description(row):
    ds_description = json.loads((Path('/'.join(row.path.parts[:4])) / 'dataset_description.json').read_text())
    print(ds)
    print(json.dumps(ds_description, indent=2))

In [181]:
good_list = []

In [182]:
# it'd be good to have more than one participant, not going to use the russome
print_ds_description(potential_ds.loc['ds000031'])

ds003673
{
  "Name": "MyConnectome",
  "License": "PDDL",
  "Authors": [
    "Poldrack RA",
    "Laumann TO",
    "Koyejo O",
    "Gregory B",
    "Hover A",
    "Chen MY",
    "Luci J",
    "Joo SJ",
    "Handwerker D",
    "Liang J",
    "Boyd R",
    "Hunicke-Smith S",
    "Simpson ZB",
    "Caven T",
    "Sochat V",
    "Shine JM",
    "Gordon E",
    "Snyder AZ",
    "Adeyemo B",
    "Petersen SE",
    "Glahn D",
    "McKay DR",
    "Curran JE",
    "Goring HHH",
    "Carless MA",
    "Blangero J",
    "Frick L",
    "Marcotte E",
    "Mumford JA"
  ],
  "Acknowledgments": "Jacy Anthis, Ashley Armenita, Adiel Carlo, Andrew Coulter, Jave Del Rosario, Dustin Fry, Kristie Garza, Kelly Jameson, Seorin Kim, Christina Leuker, Lakshya Nagar, Jacob Perez, Grace Shearrer, and Sheila Zare for assistance with MRI scanning",
  "HowToAcknowledge": "Data for the MyConnectome project were obtained from the OpenNeuro database (ds000031); also cite: Poldrack et al., (2015). Long-term neural and ph

In [183]:
# poteintially useable
print_ds_description(potential_ds.loc['ds001168'])
good_list.append('ds001168')

ds003673
{
  "Name": "A high resolution 7-Tesla resting-state fMRI test-retest dataset with cognitive and physiological measures",
  "License": "CC0",
  "Authors": [
    "Chris Gorgolewski",
    " Natacha Mendes",
    "Domenica Wilfling",
    "Elisabeth Wladimirow",
    "Claudine J. Gauthier",
    "Tyler Bonnen",
    "Florence J.M. Ruby",
    "Robert Trampel",
    "Pierre-Louis Bazin",
    "Roberto Cozatl",
    "Jonathan Smallwood",
    "Daniel S. Margulies"
  ],
  "Acknowledgements": "",
  "HowToAcknowledge": "Please cite https://doi.org/10.1038/sdata.2014.54",
  "Funding": "",
  "ReferencesAndLinks": [
    "https://doi.org/10.1038/sdata.2014.54",
    "http://fcon_1000.projects.nitrc.org/indi/CoRR/html/mpg_1.html"
  ],
  "DatasetDOI": "10.18112/openneuro.ds001168.v1.0.1",
  "BIDSVersion": "1.0.0rc3"
}


In [184]:
# macaque
print_ds_description(potential_ds.loc['ds001875'])


ds003673
{
  "Name": "TheVirtualBrain Macaque MRI",
  "BIDSVersion": "1.0.2",
  "License": "CC0",
  "Authors": [
    "Kelly Shen",
    "Joseph S. Gati",
    "Ravi S. Menon",
    "Stefan Everling",
    "Anthony R. McIntosh"
  ],
  "Acknowledgements": "",
  "HowToAcknowledge": "The following publication should be cited when using this dataset: Shen K, Bezgin G, Schirner M, Ritter P, Everling S, McIntosh AR (2018) A macaque connective for large-scale network simulations in TheVirtualBrain. bioRxiv 480905, doi: https://doi.org/10.1101/48905.",
  "Funding": [
    "Canadian Institutes of Health Research (FRN 148365)",
    "Canada First Research Excellence Fund (to BrainsCAN)",
    "J.S. McDonnell Foundation (Collaborative Research Grant 220020255)"
  ],
  "ReferencesAndLinks": [
    "Additional information on this dataset can be found in the following papers: 1) Ghahremani M, Hutchison RM, Menon RS, Everling S (2017) Frontoparietal functional connectivity in the common marmoset. Cerebral Cor

In [185]:
# poteintially useable
print_ds_description(potential_ds.loc['ds002330'])
good_list.append('ds002330')

ds003673
{
  "Name": "Neuroimaging predictors of creativity in healthy adults",
  "BIDSVersion": "1.2.0",
  "Authors": [
    "Sunavsky, A.",
    "Poppenk, J."
  ],
  "ReferencesAndLinks": [
    "https://www.sciencedirect.com/science/article/pii/S1053811919308833"
  ],
  "License": "CC0",
  "DatasetDOI": "10.18112/openneuro.ds002330.v1.1.0"
}


In [186]:
# it'd be good to have more than one participant
print_ds_description(potential_ds.loc['ds002372'])


ds003673
{
  "HowToAcknowledge": "When using the dataset, please cite the paper in the ReferencesAndLinks.",
  "Authors": [
    "Mehraveh Salehi",
    "Abigail Greene",
    "Xilin Shen",
    "An Qu",
    "David O'Connor",
    "Dustin Scheinost",
    "R. Todd Constable"
  ],
  "Name": "Yale_Single_Subject_Task_Rest30x",
  "Acknowledgements": "",
  "Funding": [
    ""
  ],
  "ReferencesAndLinks": [
    "Salehi, M., Greene, A.S., Karbasi, A., Shen, X., Scheinost, D. and Constable, R.T., 2019. There is no single functional atlas even for a single individual: Functional parcel definitions change with task. NeuroImage, p.116366."
  ],
  "BIDSVersion": "1.1.1",
  "License": "PD"
}


In [187]:
# potentially usable, though info in the dataset is pretty sparse
print_ds_description(potential_ds.loc['ds003059'])
print(Path('../data/openneuro/ds003059/sub-001/ses-PLCB/func/sub-001_ses-PLCB_task-rest_bold.json').read_text())
good_list.append('ds003059')

ds003673
{
  "Name": "Neural correlates of the LSD experience revealed by multimodal neuroimaging",
  "BIDSVersion": "1.4.0",
  "DatasetType": "derivative",
  "Authors": [
    "Robin Carhart-Harris et al."
  ],
  "HowToAcknowledge": "Please cite this paper: https://www.pnas.org/content/113/17/4853.short",
  "DatasetDOI": "10.18112/openneuro.ds003059.v1.0.0",
  "License": "CC0",
  "Funding": [
    "Walacea.com crowd-funding",
    "Safra Foundation",
    "Beckley Foundation"
  ]
}
{
    "EchoTime": 0.035, 
    "RepetitionTime": 2, 
    "TaskName": "Rest"
   
}


In [188]:
# rat
print_ds_description(potential_ds.loc['ds003520'])
#good_list.append('ds003059')

ds003673
{
  "Name": "Rat_rest_STZ",
  "BIDSVersion": "1.0.2",
  "License": "CC0",
  "Authors": [
    "Yujian Diao",
    "Ileana O Jelescu"
  ],
  "Acknowledgements": "Please acknowledge the CIBM Center for Biomedical Imaging, a Swiss research center of excellence founded and supported by Lausanne University Hospital (CHUV), University of Lausanne (UNIL) Ecole Polytechnique Federale de Lausanne (EPFL), University of Geneva (UNIGE) and Geneva University Hospitals (HUG)",
  "HowToAcknowledge": "Please cite the following articles: \n\nCatarina Tristao Pereira, Yujian Diao, Ting Yin, Analina R da Silva, Bernard Lanz, Katarzyna Pierzchala Carole Poitry-Yamate and Ileana O Jelescu, Synchronous nonmonotonic changes in functional connectivity and white matter integrity in a rat model of sporadic Alzheimers disease. NeuroImage 225 (2021) 117498. https://doi.org/10.1016/j.neuroimage.2020.117498; \n\nYujian Diao, Ting Yin, Rolf Gruetter and Ileana O Jelescu, PIRACY: An Optimized Pipeline for Func

In [190]:
# rat
print_ds_description(potential_ds.loc['ds003646'])


ds003673
{
  "Name": "An isotropic EPI database for rat brain resting-state fMRI",
  "BIDSVersion": "1.2.2",
  "License": "CC0",
  "Authors": [
    "Sung-Ho Lee",
    "Margaret A. Broadwater",
    "Woomi Ban",
    "Tzu-Wen Winnie Wang",
    "Hyeon-Joong Kim",
    "Jaiden Seongmi Dumas",
    "Ryan P. Vetreno",
    "Melissa A. Herman",
    "A. Leslie Morrow",
    "Joyce Besheer",
    "Thomas L. Kash",
    "Charlotte A. Boettiger",
    "Donita L. Robinson",
    "Fulton T. Crews",
    "Yen-Yu Ian Shih"
  ],
  "Acknowledgements": "We thank Dr. Al Johnson for sharing high resolution structural MRI data and investigators at UNC CAMRI, especially Drs. Domenic Cerri, Lindsay Walton, Martin MacKinnon, Yuncong Ma, and Li-Ming Hsu for insightful discussion.",
  "HowToAsknowledge": "",
  "Funding": [
    "National Institute of Mental Health (R01MH126518, RF1MH117053, R01MH111429, R41MH113252)",
    "National Institute on Alcohol Abuse and Alcoholism (P60AA011605, R01AA025582, K01AA025383, U24AA0200

In [192]:

print_ds_description(potential_ds.loc['ds003673'])
good_list.append('ds003673')

ds003673
{
  "Name": "Yale Resting State fMRI/Pupillometry: Arousal Study",
  "BIDSVersion": "1.2.0",
  "License": "CC0",
  "Authors": [
    "Kangjoo Lee",
    "Corey Horien",
    "David O'Connor",
    "Bronwen Garand-Sheridan",
    "Fuyuze Tokoglu",
    "Dustin Scheinost",
    "Evelyn MR Lake",
    "Todd R Constable"
  ],
  "Funding": [],
  "HowToAcknowledge": "When using the dataset, please cite the paper in the ReferencesAndLinks.",
  "EthicsApprovals": [
    "This study was approved by the Institutional Review Board at Yale University. "
  ],
  "ReferencesAndLinks": [],
  "DatasetDOI": "10.18112/openneuro.ds003673.v1.0.1"
}


In [193]:
good_list

['ds001168', 'ds002330', 'ds003059', 'ds003673']

In [204]:
good_ds = potential_ds.reset_index()
good_ds = good_ds.loc[good_ds.dataset.isin(good_list)]

nsubs = entities.groupby('dataset')[['subject']].nunique()

good_ds = good_ds.merge(nruns.reset_index().rename(columns={'run': 'nruns'}), how='left', on='dataset')
good_ds = good_ds.merge(nsubs.reset_index().rename(columns={'subject': 'nsubs'}), how='left', on='dataset')


In [205]:
good_ds

,dataset,subject,session,task,run,suffix,datatype,extension,echo,acquisition,...,desc,direction,path,x_res,y_res,z_res,tr,time,nruns,nsubs
0,ds001168,01,1,rest,1,bold,func,.nii.gz,None,fullbrain,...,None,None,../data/openneuro/ds001168/sub-01/ses-1/func/sub-01_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz,1.5,1.5,1.5,3000.0,15000.000000,2,22
1,ds002330,01,None,rest,1,bold,func,.nii.gz,None,None,...,None,None,../data/openneuro/ds002330/sub-01/func/sub-01_task-rest_run-01_bold.nii.gz,1.5,1.5,1.5,1.9,4.750000,2,66
2,ds003059,001,LSD,rest,1,bold,func,.nii.gz,None,None,...,None,None,../data/openneuro/ds003059/sub-001/ses-LSD/func/sub-001_ses-LSD_task-rest_run-01_bold.nii.gz,2.0,2.0,2.0,2.0,7.233333,3,15
3,ds003673,pa1372,None,rest,1,bold,func,.nii.gz,None,None,...,None,None,../data/openneuro/ds003673/sub-pa1372/func/sub-pa1372_task-rest_run-01_bold.nii.gz,2.0,2.0,2.0,1.0,6.833333,2,27
